# VLSI DESIGN USING Z3 SOLVER

 Vida Zahedi-vida.zahedi@studio.unibo.it
 Samral Tahirli-samral.tahirli@studio.unibo.it

In [ ]:
!pip install z3-solver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 13.0 MB/s eta 0:00:00


In [ ]:
#importing required libraries
import time
import numpy as np
from z3 import And, Or, Int,Not, Solver, sat, If, Sum , Bool,Implies,Optimize
start_time = time.time()
Num = 6 #the number of instance we want to find the solution for.     
Inputs = []
#reading the file and seperating each row.
with open( InstanceName , 'r') as Sample:
    for line in Sample:
        Inputs.append(line.replace('\n',''))
#Making a 2 dimensional list of the Inputs in str format.       
Inputs=[line.split(' ') for line in Inputs]
#print(Inputs)

# we must change the inputs from string to integer.
# Defining the inputs and variables. 
Width   = int(Inputs[0][0]) 
Number  = int(Inputs[1][0]) 
Widths  = [int(line[0]) for line in Inputs[2:]] 
Heights = [int(line[1]) for line in Inputs[2:]]
#defining the upper boundary for the height of plate => calling it MaxHeight.
MaxHeight = sum(Heights) 
#defining the lower boundary for the height of plate => calling it MinHeight.
MinHeight =np.divide((np.sum(np.multiply(Widths,Heights))) , Width)
#Defining the output variables which is the x and y coordinate for each piece.
x_ith =[]
for i in range(Number):
    x_ith.append(Int("x_%d" % i) )

y_ith =[]
for i in range(Number):
    y_ith.append(Int("y_%d" % i)) 

def FindM(ele):
  Max = ele[0]
  for i in ele[1:]:
    Max = If(i > Max, i, Max)
  return Max    
sorted_circuits = sorted(zip(Widths, Heights), key=lambda x: x[0]*x[1], reverse=True)
Widths1, Heights1 = zip(*sorted_circuits)    
OptHeight = FindM([y_ith[i] + Heights[i] for i in range(Number)])
#definig diffn 
def diffn(x,y,dx,dy):
    n = len(x)
    overlap = [
        Or(x_ith[i] + Widths[i]  <= x_ith[j],
           x_ith[j] + Widths[j]  <= x_ith[i],
           y_ith[i] + Heights[i] <= y_ith[j],
           y_ith[j] + Heights[j] <= y_ith[i]) 
           for i in range(Number) 
        for j in range(Number) if i < j]
    return overlap
overlap1 = diffn(x_ith,  y_ith, Widths, Heights)
overlap2 = diffn(x_ith,  y_ith, Widths1, Heights1)

Bound = [] 
for i in range(Number):
    Bound.append(And(x_ith[i] >= 0, 
        y_ith[i] >= 0,
        x_ith[i] + Widths[i] <=Width))            
          


In [ ]:
# Creating an optimization object
opt = Optimize()

# Adding constraints
#opt.add(x_ith[0] == 0, y_ith[0] == 0)
opt.add(overlap1)
opt.add(overlap2)
opt.add(Bound)
# Adding an optimization goal
opt.minimize(OptHeight)
# Checking if the optimization is satisfiable
if opt.check() == sat:
    model = opt.model()
    # Extracting the values of x and y from the optimization model
    x_vals = [model.eval(x_ith[i]).as_long() for i in range(Number)]
    y_vals = [model.eval(y_ith[i]).as_long() for i in range(Number)]
    print("x_vals: ", x_vals)
    print("y_vals: ", y_vals)
    print("Minimized height: ", model.eval(OptHeight).as_long())
else:
    print("The optimization is not satisfiable")  
timing = (time.time() - start_time)
print(timing)

x_vals:  [7, 7, 0, 10, 10, 0, 3, 3, 3]
y_vals:  [6, 9, 8, 0, 6, 0, 10, 6, 0]
Minimized height:  13
4.56686806678772
